### 简介
在raft出现之前, 分布式一致性通常使用paxos来解决; 但是raft比paxos更能被人理解, 而且为构建分布式系统提供了更好的基础.  
raft设计的主要目标就是可理解性, 采取了一些列方法来提高算法的可理解性  
* 分解 - (decomposition)  
  raft把问题分解成几个场景下的子问题: learder选举, log 副本, 安全性  
* 降低状态空间 - (state space reduction)   
  相比paxos, raft减少了不确定性的场景和不同server达到不一致性的因素  
  
### 复制状态机器 - Replicated state machine
&nbsp;&nbsp;&nbsp;&nbsp;一致性算法通常都会放置在 Replicated state machine 场景下讨论; 状态机器 (state machines) 是一组 server 集合, 这组 server 集合可以在 server 间计算出相同的状态拷贝, 并且在某些机器宕机后仍能继续操作; 复制状态机通常有2个应用:    
* 复制状态机通常用于解决分布式系统的 fault tolerance , 例如Hdfs. 
* 使用复制状态机管理 leader 选举和存储配置信息, 例如zookeeper

复制状态机通常使用复制log实现 (reolicated log) , 其实现架构为:  
```
一致性算法管理client方发送的 command 的 replicated log , 复制状态机对这些 command 执行相同顺序的处理, 因此, 这些状态机能够得到相同的输出
```  
&nbsp;&nbsp;&nbsp;&nbsp;综上所述, 一致性算法的工作就是保证 replicated log 的一致性. 首先, 某个一致性模块在一台 server 上接收来自 client 的 command , 并将 command 加入到自己 log 中; 然后, 这个一致性模块和处于其它 server 上的一致性模块通信, 确保每个 server 上的 log 最终都会以相同的顺序记录相同的请求. 一致性算法通常需要以下几个模块:  
* 确保安全  
  要在所有非拜占庭条件下(网络延迟,分区,丢包,消息重复, 消息重排序)返回正确的结果 
* 只要大多数 server 之间可以交互信息, 并且可以和 client 交互信息, 这个服务就是可用的
* 不依赖于时间来确保 log 的一致性
* 只要集群中的大多数对 command 作出相应就表示该 command 完成, 即小部分比较慢的节点不会影响整个系统的性能

### Raft一致性算法

&nbsp;&nbsp;&nbsp;&nbsp;首先, Raft会选举出一个leader, 该 leader 全权负责管理replicated log. 该 leader 接收来自于 client 的 日志条目, 并在其它 server 节点备份该日志条目, 并告知这些server何时可以安全的应用这些日志条目来更改状态. 通过 leader 来简化复制日志的管理, 比如数据流只能是从 leader 流向 其它 server, 当leader宕机失联后, 会选举一个新的 leader   

|术语|解释|
| :-----| ----- |
|term number|期限号|
|split vote|投票分裂<br>(大多数follpwer同时成为candidate, 都推举自己导致无法选出leader)|
|election timeout|选举超时<br>(follower在一段时间内每收到来自leader的AppendEntries RPC请求, 认为集群中没有合法的leader)|
|log entry|日志条目|




#### 5.1 raft basics
&nbsp;&nbsp;&nbsp;&nbsp;1个raft集群包含多个 server , 通常拿5个举例, 5台 server 的集群可以容忍2台失败; 任一时刻, 每个 server 都处于以下3种状态其中之一: `leader , follower , candidate`
* Follower是被动的(passive): 只会响应来自于leader或candidate的请求  
* Leader: Leader处理所有的客户端请求, 如果客户端连接到了follower, 则会被重定向到leader  
* Candidate: 用于选举1个新的leader

raft把时间分割成任意长度的期限(term), 期限(term)是连续的正数. 每一个term都从选举开始:
* 如果一个candidate赢得选举, 它会在接下来的term中成为leader;
* 某些情况下, 选举会以投票分裂结束(split vote), 此时任期会以无leader结束, 而1个新的任期会马上开启. raft保证在任期内最多只有1个leader

term(期限)相当于逻辑时钟, 每个server都会保存当前的期限数字(term number), 期限数字会随着时间的流逝而单调递增
* server 之间相互通信时, 当一个server发现自己的期限号小于其它server的期限号时, 就回吧自己的期限号更新为较大的值 
* 如果1个candidate或leader发现自己的期限号过期了, 会立刻转变为follower
* 如果一个 server 收到1个过期期限号的request , 该 server 会拒绝这个请求

raft使用RPC在不同server之间进行通信, 一般包括两种RPC:
* RequestVote RPC : 被candidate产生, 用于选举
* AppendEntries RPC : 被leader产生, 用于复制日志项和提供心跳

#### 5.2 leader选举
raft通过心跳机制触发leader选举. server 启动时会以follower身份出现. server会保持 follower 身份只要它从leader/candidate中收到合法的RPC
* leader 周期性的发送  AppendEntries RPC 到所有的 follower 来维持自己的 leader 身份; 如果某个 follower 在一段时间内没有收到 leader 的心跳包, 则产生选举超时(election timeout), 此时该 follower 认为集群中没有 leader 从而触发新的 leader 选举  

follower如何开启选举呢
* 开始选举时, 一个 follower 会增加自己当前的期限, 并把自己提升成candidate, 然后推举自己为 leader 并发送 RequestVote RPC 请求到其它 serve . 除非以下三件事有一件发生, 否则持续维持自己是 candidate 状态 : 
    * 它赢得了选举
    * 其它 server 已经成为 leader
    * 一段时间后, 仍没有产生获胜者
    
candidate如何赢得选举呢
* 一个candidate赢得选举, 只要它收到了集群中大多数server对同一个期限(term)的投票. 每个 server 都最多为1个 candidate 投票, 使用"谁先来先投谁""的办法. 一旦某个 candidate 赢得了选举, 就会和其它 server 发送心跳建立自己的leader权限并阻止进行新的选举
* 一个candidate在等待其它 server 的投票过程中, 可能会收到其它 server 发送来的 AppendEntries RPC 请求.
    * 如果该 AppendEntries RPC 中的期限大于等于自己的期限, 则该 candidate 意识到已有合法的 leader 产生, 于是自己切换回 follower 状态
    * 如果该 AppendEntries RPC 中的期限小于自己的期限, 则该 candidate 拒绝此次 RPC 请求, 并维持 candidate 状态  
    
第三种情况是, 一个 candidate 在此次选举中既没有成功也没有失败  
* 如果很多 follower 在同一时间都成为了 candidate , 投票就会产生分裂导致不存在一个 candidate 可以获得大多数的投票, 此时所有的 candidate 会继续增加自己的期限(term) 并开启新一轮的选举, 如果没有额外的度量手段, 这种"分裂投票"会无休止的重复下去
* raft使用"随机选举超时"来减少投票分裂出现, 并确保即使出现投票分裂问题, 也能被快速解决     
    * 减少"split vote"的发生:  
      随机的从一个范围内选择"选举超时"时间, 例如150~300毫秒, 这使得大多数情况下只有1个 follower 发生选举超时; 他会在其它 server 超时之前赢得选举并发送心跳
    * 同样的做法也用来解决"split vote"  
      当split vote发生后, candidate 也会选择一个随机的时间间隔等待投票结果, 在这个随机的时间后, 若仍没有leader被选出来才开启新一轮的选举
      
#### 5.3 日志复制 Log Replication
&nbsp;&nbsp;&nbsp;&nbsp;一旦有某个 leader 选举出来后, 它开始服务 client 的请求. leader 将 client 的 command 作为新的 log entry 追加到自己的 log 中, 然后发送 AppendEntries RPC 到其它 server 中进行log 复制. 当 entry 被安全的复制后, leader 将应用这个 command 并向 client 返回结果.   
&nbsp;&nbsp;&nbsp;&nbsp; 每个 log entry 都包含 command 和 leader 的期限号, 其次 log entry 还包含一个整数索引表示该条目在日志中的位置   
&nbsp;&nbsp;&nbsp;&nbsp; leader 决定什么时候可以安全的执行 log entry 中的 command, 就是这个entry处于 "committed" 状态. raft保证committed状态的日志条目最终会被所有可用的状态机执行. 一个日志条目被称作committed的: 只要leader创建了该条目, 并将该条目复制到了进群中的绝大多数机器. leader会记录处于committed状态的日志条目的index的最大值, 并将index的最大值放入 AppendEntries RPC 中, 一旦 follower 发现日志条目是 committed 的, 就会将这个条目的 command 应用到自己的本地机器中. 
&nbsp;&nbsp;&nbsp;&nbsp;raft的日志复制机制有以下两个特点: 
* 如果不同日志里的2个条目, 有相同的index和term, 则2个条目中存放着相同的 command  
* 如果不同日志里的2个条目, 有相同的index和term, 则该条目之前的所有条目也相同

现在还遗留一个问题, 如果新一轮选举后, 选出的 leader 中的 log 只保存了客户端发来的部分command, 即新leader的 log 是残缺的, 此时会发生 leader 和 follower 中日志不一致的结果. raft简化了这种情况下的处理: 保证数据流只能从 leader 发往 follower, 即 follower 中冲突的 log entry 会被 leader中的 log entry覆盖
* 要使得follower的日志和leader的日志进入一致的状态, leader必须找到两者最后达到一致的地方, 并让follower 删除这个地方之后的本地日志. 所有的这些操作会在 AppendEntries RPC 的一致性检查中处理. leader 对每个 follower 维护一个 nextindex, 表示要发给 follower 的下一个日志条目的 index .
    * 当 leader 第一次成为 leader 时, 会初始化所有的 nextIndex 为自己日志条目的最后一条+1, 然后发送 AppendEntries RPC 到每个 fllower , 在follower收到请求进行一致性检查时, 如果发现上个 leader 的上个日志条目和自己本地的不同, 则 follower 会发出失败响应, leader 在收到这个失败响应后, 将 lastIndex 减小再次构建 AppendEntries RPC, 直到follower直到和自己相同的最后1个日志条目, 让 leader 得知正确的nextIndex   
    
这种处理 leader 和 follower 日志不一致的办法并不会导致客户端的 command 丢失, 原因是在 leader 选举时增加的一些安全性检查条件, 如下面所述

#### 5.4 安全性检查  
* 选举限制    
  raft使用一种简单的办法, 确保新产生的 leader 中包含往期所有term的日志, 不会让日志条目从 follower 流向 leader. 即让 RequestVote RPC 中包含 candidate 自己的日志信息: 包含最后的term 和 log index. term 比较靠后的日志比较新, term相同时, log index 更大的比较新.
  
#### 5.5 follower和candidate的失败
follower和candidate的失败策略是一样的, 如果 follower 或 candidate 宕机, 会导致发送给他们的RPCs失败, raft简单的通过无限的重试, 一旦 follower 或 candidate 重启后, 那么他会再次收到同样的请求. 因此, 要求raft 中的 RPC 时幂等的, 这很好实现, 比如: AppendEntriesRPC, 如果 follower 发现自己的日志中已包含该条目, 则忽略这个请求